In [39]:
import pandas as pd
import numpy as np
import re

In [40]:
data_path = '/data/ephemeral/home/data/train.csv'
df = pd.read_csv(data_path)

In [41]:
def classificate_by_label(df):
    labels = df['target'].unique() 
    labels = np.sort(labels)

    label_dfs = {label: df[df['target'] == label] for label in labels}

    for label, data in label_dfs.items():
        print(f"Target {label}: {len(data)} rows")
     
    return label_dfs
        
label_dfs = classificate_by_label(df)   

Target 0: 397 rows
Target 1: 410 rows
Target 2: 388 rows
Target 3: 385 rows
Target 4: 406 rows
Target 5: 419 rows
Target 6: 395 rows


In [42]:
def get_length_excluding_spaces(text):
    # 공백을 제외한 문자열의 길이 계산
    length_excluding_spaces = len(text.replace(" ", ""))
    return length_excluding_spaces

df['text_length'] = df['text'].apply(get_length_excluding_spaces)

In [43]:
# 특수문자 비율 계산 함수
def calculate_special_char_ratio(row):
    text = row['text']
    total_length = row['text_length']
    if total_length == 0:
        return 0
    special_chars = re.findall(r'[^\w\s]', text)
    return len(special_chars) / total_length

# 영어 문자 비율 계산 함수
def calculate_english_char_ratio(row):
    text = row['text']
    total_length = row['text_length']
    if total_length == 0:
        return 0
    english_chars = re.findall(r'[A-Za-z]', text)
    return len(english_chars) / total_length

# 숫자 비율 계산 함수
def calculate_number_ratio(row):
    text = row['text']
    total_length = row['text_length']
    if total_length == 0:
        return 0
    numbers = re.findall(r'[0-9]', text)
    return len(numbers) / total_length

# 각각의 열에 함수 적용
df['special_char_ratio'] = df.apply(calculate_special_char_ratio, axis=1)
df['english_char_ratio'] = df.apply(calculate_english_char_ratio, axis=1)
df['number_ratio'] = df.apply(calculate_number_ratio, axis=1)

print(df)


                       ID                               text  target  \
0     ynat-v1_train_00000   정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4   
1     ynat-v1_train_00001        K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3   
2     ynat-v1_train_00002             m 김정) 자주통일 새,?r열1나가야1보       2   
3     ynat-v1_train_00003      갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩       5   
4     ynat-v1_train_00004       pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6   
...                   ...                                ...     ...   
2795  ynat-v1_train_02795  트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도       6   
2796  ynat-v1_train_02796        삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠       2   
2797  ynat-v1_train_02797   텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보       4   
2798  ynat-v1_train_02798    인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정       1   
2799  ynat-v1_train_02799               지능정보사회 대비 국가 종합대책 마련       4   

      text_length  special_char_ratio  english_char_ratio  number_ratio  
0              25            0.120000            0.280000    

In [44]:
df['text'] = df['text'].str.replace('…', ' ', regex=False)

In [45]:
text_value = df[df['ID'] == 'ynat-v1_train_02798']['text'].iloc[0]

# 각 문자와 유니코드 값을 출력
print([(char, ord(char)) for char in text_value])

[('인', 51064), ('터', 53552), ('뷰', 48624), (' ', 32), ('류', 47448), ('현', 54788), ('진', 51652), (' ', 32), ('친', 52828), ('구', 44396), ('에', 50640), ('게', 44172), (' ', 32), ('안', 50504), ('타', 53440), (' ', 32), ('맞', 47582), ('는', 45716), (' ', 32), ('것', 44163), (' ', 32), ('싫', 49899), ('어', 50612), ('해', 54644), (' ', 32), ('승', 49849), ('부', 48512), ('는', 45716), (' ', 32), ('냉', 45257), ('정', 51221)]


In [46]:
## 현재 문장에서 ascii의 비율과, ascii의 비율 중 영어와 대문자가 있는 경우에 대한 확인.
def calculate_ascii(text):
    # 공백을 제외한 ASCII 문자들
    ascii_chars = [char for char in text if ord(char) < 128 and not char.isspace()]
    ascii_count = len(ascii_chars)
    ascii_ratio = ascii_count / len(text)

    # ASCII 문자들이 영어에 대한 
    english_chars = [char for char in ascii_chars if char.isalpha()]
    has_english = len(english_chars) > 0
    # 영어 관련 체크 영어 관련 내용이 있을경우
    is_english_only = all(char.isalpha() for char in ascii_chars) if has_english else False
    # 대문자 관련 계산 (영어가 하나라도 있을 때만 체크)
    is_all_uppercase = all(char.isupper() for char in english_chars) if has_english else False
    uppercase_count = sum(1 for char in ascii_chars if char.isupper())
    uppercase_ratio = uppercase_count / len(ascii_chars) if ascii_chars else 0
    
    return ascii_count, is_english_only, is_all_uppercase, ascii_ratio, uppercase_ratio

In [47]:

df['text'] = df['text'].str.replace('...', ' ').str.replace('…', ' ')
df['length'] = df['text'].map(len)
df['ascii_count'], df['is_english_ascii'], df['is_all_uppercase'], df['ascii_ratio'], df['uppercase_ratio'] = zip(*df['text'].apply(calculate_ascii))

# 결과 확인
print("ASCII 분석 결과:")
df[['text', 'ascii_count', 'is_english_ascii', 'is_all_uppercase', 'ascii_ratio', 'uppercase_ratio']].head()

ASCII 분석 결과:


,text,ascii_count,is_english_ascii,is_all_uppercase,ascii_ratio,uppercase_ratio
0,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,13,False,False,0.406250,0.307692
1,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,16,False,False,0.592593,0.375000
2,"m 김정) 자주통일 새,?r열1나가야1보",7,False,False,0.318182,0.000000
3,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,3,False,False,0.103448,0.000000
4,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,9,False,False,0.321429,0.333333


In [ ]:
df['english_count'] = df['text'].str.findall(r'[A-Za-z]+').str.len()

r가d나d다 3
r가dd다 2
rdd다 1

In [ ]:
cond1 = df['ascii_ratio'] >= 0.2
cond2 = df['uppercase_ratio'] > 0.8
sep_eng_count_cond = df['english_count'] < 4

cond2_and_sep_eng_count = df[cond2 & sep_eng_count_cond]

overlapping_indices = cond2_and_sep_eng_count.index

filtered_df = df[cond1 & ~df.index.isin(overlapping_indices)]


In [38]:
len(filtered_df)

1619

In [21]:
count_summary = df['english_count'].value_counts().sort_index()

print(count_summary)

english_count
0     923
1     254
2     218
3     294
4     312
5     326
6     247
7     129
8      66
9      19
10      8
11      3
12      1
Name: count, dtype: int64


In [22]:
filtered_df = df[df['english_count'] >= 4][['ID', 'text', 'target']].copy

In [24]:
filtered_df

<bound method NDFrame.copy of                        ID                              text  target
0     ynat-v1_train_00000  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4
1     ynat-v1_train_00001       K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3
4     ynat-v1_train_00004      pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6
10    ynat-v1_train_00010       oi 매력 R모h츠a열#w3약 >l·주가 고Q/진       5
21    ynat-v1_train_00021      KIA I수단·팬nI께하는:호kQ4M족 한마5 S최       1
...                   ...                               ...     ...
2784  ynat-v1_train_02784   고려·vq 5|째 결승 격돌la~일,%3대o농구 챔피Tw       1
2786  ynat-v1_train_02786         텅 빈 }실 선생님+홀K Y|q% T+X C1       3
2787  ynat-v1_train_02787   13이 노바 라이n2·미J어패0 T3 10 결G상품% *       6
2792  ynat-v1_train_02792       높`X#E율 }BO Q"[/선수 몸값 상승 CAO       1
2797  ynat-v1_train_02797  텔레그램+한D 등h亞서 2시간H다운 C버T정gf39종!2보       4

[1111 rows x 3 columns]>

In [18]:
filtered_high_ascii = high_ascii[high_ascii['english_count'] >= 4][['ID', 'text', 'target']].copy

In [19]:
filtered_high_ascii

<bound method NDFrame.copy of                        ID                                     text  target
561   ynat-v1_train_00561                물로 덮인 U터월드S행성에도 G0체H존PEF능       4
710   ynat-v1_train_00710            문X통& 빠E O전선언 바람직J다는 P 대V로 공감대       2
1776  ynat-v1_train_01776  LGU 5G SA 상용화 준비 SA 기술 NSA 코어 장비에 연동 검증       5
1892  ynat-v1_train_01892                    LCJLR국FjU권A어^이 백S  시상       0
2704  ynat-v1_train_02704               L자Mk 삼성생D OK저축은P에 완승 연패 탈출       1>

In [66]:
label_dfs = classificate_by_label(high_ascii)  

Target 0: 225 rows
Target 1: 232 rows
Target 2: 229 rows
Target 3: 232 rows
Target 4: 232 rows
Target 5: 232 rows
Target 6: 226 rows


In [24]:
num = 6
label_dfs[num]['text']

total_rows = len(label_dfs[num])
chunk_size = 20

for i in range(0, total_rows, chunk_size):
    chunk = label_dfs[num].iloc[i:i + chunk_size]
    print(f"\n=== {i+1}~{min(i+chunk_size, total_rows)}번째 데이터 ===")
    print(chunk[['text']])
    
    # 다음 20개를 보려면 엔터를 누르세요
    input("\n다음 20개를 보려면 Enter를 누르세요...")


=== 1~20번째 데이터 ===
                                   text
4          pI美대선I앞두고 R2fr단 발] $비해 감시 강화
50   美MBA[여성 비율x계속 x가4주$E19a대 U입생 중Ym,%
58      한#M2 !는 유`8 치료제 오B솔 美 임{ 3a 본격화
59       아사w 트K프 외교S험k없다고 i5오바I도]KR<8였-
67       x콩-P면금[T 나%\ g트=물J1h나고 지>철 %기고
86     ~리비{3Qn'9 신분 변:+허용 의료u서로 사전w입p해야
107           프랑스 마N}의 이I 대\- G7 ;청설 c인
134       코오롱플라스틱 1분기 연결 영업익 58억원 138%↑
159               中[정u 지방W민0투Ad실태K6L}착5
172           美 미네소타r州H람M장 입구H 2격d 명 부상
174    \페이오k중동서".]Vx국fr동]Cr구 이란B고. 강L종Q
187  반기문m유p{e 관저에>리2창 부부 초대 만찬 2년전|약속지켜
198      버닝썬 게이트 다룬 SBS 그것이 알고 싶다 11.2％
206       이kDr|크>G리아 친이T0시설 >격 u]라J에 경/
216            k@ w고지;~ 반정,시/에 美X사주한 1)
237          출마금cyj슈9웡 홍'!.t(!%원하면 투표해t
243            ~:자베스 여,+정치권에fS4/트 타협 촉구
244       UAG|'전 =기-맞먹c 사K E양)i;전소 /lco
246        印공? 파키스탄 내 w$ P@ +o ,등 K고조종합
299    푸4 살만i사우디"Q왕과eo= ^유시< >조 지속 합의종합

=== 21~40번째 데이터 ===
                                  text
303       아랍권 반D란"정상회의 5춰 (\s& 이란 맹9종합
317       탈북 도와t;한국계f영b7#-왕s 국가공:훈장 o아
34

In [21]:
english_label_dfs = classificate_by_label(is_english_df) 

Target 0: 29 rows
Target 1: 25 rows
Target 2: 19 rows
Target 3: 22 rows
Target 4: 18 rows
Target 5: 24 rows
Target 6: 24 rows


In [43]:
def is_continuous_english(text):
    """
    주어진 텍스트에 연속된 영어가 연속적으로 포함되어 있는지 확인.
    """
    # 정규표현식: 3글자 이상의 연속된 영어 단어 찾기
    abnormal_pattern = re.compile(r'[a-zA-Z]{2,}')
    
    # 정규식에 매칭되는 영어 단어 찾기
    matches = abnormal_pattern.findall(text)
    
    if matches:
        # 연속적인 영어 단어가 있을 경우 True 반환
        return matches
    return False

is_english_df['continuous'] = is_english_df['text'].apply(is_continuous_english)
no_contigus_english_df = is_english_df[is_english_df['continuous']==False].copy()
no_contigus_english_df[['ID','text', 'target','continuous']]

,ID,text,target,continuous
18,ynat-v1_train_00018,개R전 연w정연H 작가,0,False
38,ynat-v1_train_00038,마틴 우리카드 코치로 V리그 복귀 한국 좋아요,5,False
90,ynat-v1_train_00090,서울문고 N풍그U에 인수 f라인서점 교m영풍y양강d도로,0,False
139,ynat-v1_train_00139,떼었S 붙였다 액세서리Y 스마C폰 업U레이드,4,False
203,ynat-v1_train_00203,j우증권 오리X 중D 성장둔s 목표가↓,5,False
312,ynat-v1_train_00312,朴대통령 한불 비즈니스포럼·K콘 참석 파리서 경제·문화외교,0,False
357,ynat-v1_train_00357,주말 N 여행 강원권 남한강축제서 무더위 날리자 속초는 피서철 이벤트,1,False
601,ynat-v1_train_00601,美 매d g츠버그 강정호와 y별p T,6,False
920,ynat-v1_train_00920,v시H 축출후 수단 군부정권 이끄N 아우w는,6,False
1076,ynat-v1_train_01076,프로축구 비디오판독 올해 K리그 챌린지로 확대할 듯,0,False


In [40]:
filtered_no_contigus_english_df = no_contigus_english_df[no_contigus_english_df['continuous_set'].apply(lambda x: {'tuK'}.issubset(x))]
print(filtered_no_contigus_english_df)

                       ID                              text  target  length  \
1792  ynat-v1_train_01792  현tuK수 김천웅 한국인h최초로 k스라I E체바무용단 입단       0      32   

      ascii_count  is_english_ascii  is_all_uppercase  ascii_ratio  \
1792            7              True             False      0.21875   

      uppercase_ratio continuous continuous_set  
1792         0.428571      [tuK]          {tuK}  


In [39]:
all_words = sum(no_contigus_english_df['continuous'].tolist(), [])

# 3. 중복 제거 (필요한 경우)
unique_words = list(set(all_words))
unique_words

['UEFA',
 'LPG',
 'RCEP',
 'SBS',
 'USTR',
 'tbs',
 'APTLD',
 'BMW',
 'CPU',
 'ISA',
 'SKT',
 'SKB',
 'ELS',
 'CCO',
 'KGC',
 'WTO',
 'MLB',
 'SDI',
 'IBK',
 'MWC',
 'NHK',
 'AFC',
 'ETRI',
 'OECD',
 'tuK',
 'KISA',
 'CEO',
 'KBO',
 'MVP',
 'CSD',
 'MBC',
 'IoT',
 'URL',
 'NBIoT',
 'KEB',
 'KIST',
 'KBS',
 'USC',
 'LGU',
 'AfD',
 'KAIST',
 'EBS',
 'FFP',
 'SNS',
 'EPL',
 'APT',
 'GSMA',
 'UCLA',
 'TPP',
 'UAE',
 'UCC',
 'NEW',
 'ICT',
 'ACM',
 'WON',
 'NBA',
 'RFA',
 'SERICEO']